# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Benisty  ->  M. Benisty  |  ['M. Benisty']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']
A. Dumont  ->  A. Dumont  |  ['A. Dumont']
J. Li  ->  J. Li  |  ['J. Li']
T. Henning  ->  T. Henning  |  ['T. Henning']
A. Hughes  ->  A. Hughes  |  ['A. Hughes']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Arxiv has 52 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.08029


extracting tarball to tmp_2504.08029... done.


M. Benisty  ->  M. Benisty  |  ['M. Benisty']


Issues with the citations
list index out of range
Retrieving document from  https://arxiv.org/e-print/2504.08030
extracting tarball to tmp_2504.08030...

...

extracting tarball to tmp_2504.08387...

 done.
Retrieving document from  https://arxiv.org/e-print/2504.08436


extracting tarball to tmp_2504.08436...

 done.
  0: tmp_2504.08436/natnotes.tex, 332 lines
  1: tmp_2504.08436/natbib.tex, 96 lines
  2: tmp_2504.08436/Manuscript0411.tex, 766 lines
  3: tmp_2504.08436/aassymbols.tex, 579 lines


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 4 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2504.08436/Manuscript0411.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.08029-b31b1b.svg)](https://arxiv.org/abs/2504.08029) | **Coordinated Space and Ground-Based Monitoring of Accretion Bursts in a Protoplanetary Disk: Establishing Mid-Infrared Hydrogen Lines as Accretion Diagnostics for JWST-MIRI**  |
|| B. M. Tofflemire, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2025-04-14*|
|*Comments*| *Accepted to ApJ. 16 pages, 7 Figures, 3 Tables, 2 Appendices*|
|**Abstract**|            In this paper, we establish and calibrate mid-infrared hydrogen recombination lines observed with JWST as accretion tracers for pre-main-sequence stars that accrete from circumstellar disks. This work is part of a coordinated, multi-observatory effort that monitored the well-known binary system DQ Tau over three orbital periods, capturing its periodic accretion bursts. In this first paper, we present 9 epochs of MIRI-MRS spectra with near-simultaneous LCO photometry and VLT X-Shooter spectroscopy. This program caught exceptional accretion variability, spanning almost two orders of magnitude between the peak of the first periastron accretion burst and the following quiescent phases. The MIRI spectra show H I line luminosities that vary in step with the accretion-luminosity time series measured with LCO and X-Shooter. The tight correlation with accretion and the large line widths, which MIRI resolves for the first time, support an accretion-flow origin for mid-infrared H I transitions. Combining these three exceptional datasets, we derive accurate relations between mid-infrared line and accretion luminosities for three H I transitions (10-7, 7-6, 8-7), and improve upon a previous relation based on Spitzer spectra. These new relations equip the community with a direct measurement of the accretion luminosity from MIRI-MRS spectra. A MIRI-derived accretion luminosity is fundamental for time-domain chemistry studies, as well as for studies of accretion in embedded/distant sources that are currently inaccessible in the optical. With these new relations, we provide accretion luminosities for an archival sample of 38 MRS spectra of protoplanetary disks published to date.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.08034-b31b1b.svg)](https://arxiv.org/abs/2504.08034) | **X-ray Constraints on Wandering Black Holes in Stripped Galaxy Nuclei in the Halo of NGC 5128**  |
|| S. L. Feyan, et al. -- incl., <mark>A. Dumont</mark> |
|*Appeared on*| *2025-04-14*|
|*Comments*| *13 pages, 3 figures, 2 tables. Accepted for publication in ApJ*|
|**Abstract**|            A subset of galaxies have dense nuclei, and when these galaxies are accreted and tidally stripped, the nuclei can masquerade as globular clusters in the halos of large galaxies. If these nuclei contain massive central black holes, some may accrete gas and become observable as active galactic nuclei. Previous studies have found that candidate stripped nuclei rarely host luminous X-ray sources, but these studies were typically restricted to both the most massive candidate nuclei and the most luminous X-ray sources. Here we use new and archival Chandra and XMM-Newton data to search for X-ray emission in a near-complete sample of massive globular clusters and candidate stripped nuclei in the nearest accessible elliptical galaxy, NGC 5128. This sample has the unique advantage that the candidate stripped nuclei are identified dynamically via elevated mass-to-light ratios. Our central result is that 5/22 ($23^{+11}_{-6}$%) of the candidate stripped nuclei have X-ray sources down to a typical limit of $L_X \sim 5 \times 10^{36}$ erg s$^{-1}$, a fraction lower than or comparable to that among massive clusters with normal mass-to-light ratios (16/41; $39^{+8}_{-7}$%). Hence we confirm and extend the result that nearly all X-ray sources in stripped nuclei are likely to be X-ray binaries rather than active galactic nuclei. If the candidate stripped nuclei have black holes of typical masses $\sim 2 \times 10^{5} M_{\odot}$ needed to explain their elevated mass-to-light ratios, then they have typical Eddington ratios of $\lesssim 2 \times 10^{-6}$. This suggests that it will be challenging to conduct an accretion census of wandering black holes around even nearby galaxies.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.08363-b31b1b.svg)](https://arxiv.org/abs/2504.08363) | **The CARMENES search for exoplanets around M dwarfs. Cluster analysis of signals from spectral activity indicators to search for shared periods**  |
|| J. Kemmer, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-04-14*|
|*Comments*| *33 pages (16 pages main text), 14 figures, accepted for publication in Astronomy & Astrophysics*|
|**Abstract**|            A multitude of spectral activity indicators are routinely computed nowadays from the spectra generated as part of planet-hunting radial velocity surveys. Searching for shared periods among them can help to robustly identify astrophysical quantities of interest, such as the stellar rotation period. However, this identification can be complicated due to the fact that many different peaks occurring in the periodograms. This is especially true in the presence of aliasing and spurious signals caused by environmental influences affecting the instrument. Our goal is to test a clustering algorithm to find signals with the same periodicity, (i.e. with the stellar rotation period) in the periodograms of a large number of activity indicators. On this basis, we have looked to evaluate the correlations between activity indicators and fundamental stellar parameters. We used generalised Lomb-Scargle periodograms to find periodic signals in 24 activity indicators, spanning the VIS and NIR channels of the CARMENES spectrograph. Common periods were subsequently determined by a machine learning algorithm for density-based spatial clustering of applications with noise (DBSCAN). The clustering analysis of the signals apparent in the spectral activity indicators is a powerful tool for the detection of stellar rotation periods. It is straightforward to implement and can be easily automated, so that large data sets can be analysed. For a sample of 136 stars, we were able to recover the stellar rotation period in a total of 59 cases, including 3 with a previously unknown rotation period. In addition, we analysed spurious signals frequently occurring at the period of one year and its integer fractions, concluding that they are likely aliases of one underlying signal. Furthermore, we reproduced the results of several previous studies on the relationships between activity indicators and the stellar characteristics.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.08030-b31b1b.svg)](https://arxiv.org/abs/2504.08030) | **ELVES-Dwarf I: Satellites Systems of Eight Isolated Dwarf Galaxies in the Local Volume**  |
|| <mark>J. Li</mark>, et al. |
|*Appeared on*| *2025-04-14*|
|*Comments*| *Submitted to ApJ. Comments are welcome!*|
|**Abstract**|            The satellite populations of Milky Way--mass systems have been extensively studied, significantly advancing our understanding of galaxy formation and dark matter physics. In contrast, the satellites of lower-mass dwarf galaxies remain largely unexplored, despite hierarchical structure formation predicting that dwarf galaxies should host their own satellites. We present the first results of the ELVES-Dwarf survey, which aims to statistically characterize the satellite populations of isolated dwarf galaxies in the Local Volume ($4<D<10$~Mpc). We identify satellite candidates in integrated light using the Legacy Surveys data and are complete down to $M_g\approx -9$ mag. We then confirm the association of satellite candidates with host galaxies using surface brightness fluctuation distances measured from the Hyper Suprime-Cam data. We surveyed 8 isolated dwarf galaxies with stellar masses ranging from sub-Small Magellanic Cloud to Large Magellanic Cloud scales ($10^{7.8} < M_\star^{\rm host}<10^{9.5}\, M_\odot$) and confirmed 6 satellites with stellar masses between $10^{5.6}$ and $10^{8} \, M_\odot$. Most confirmed satellites are star-forming, contrasting with the primarily quiescent satellites observed around Milky Way--mass hosts. By comparing observed satellite abundances and stellar mass functions with theoretical predictions, we find no evidence of a "missing satellite problem" in the dwarf galaxy regime.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.08032-b31b1b.svg)](https://arxiv.org/abs/2504.08032) | **Counting Little Red Dots at $z<4$ with Ground-based Surveys and Spectroscopic Follow-up**  |
|| Y. Ma, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-04-14*|
|*Comments*| *15 pages, 8 figures, submitted to ApJL. Comments are welcome*|
|**Abstract**|            Little red dots (LRDs) are a population of red, compact objects discovered by JWST at $z>4$. At $4<z<8$, they are roughly 100 times more abundant than UV-selected quasars. However, their number density is uncertain at $z<4$ due to the small sky coverage and limited blue wavelength coverage of JWST. We present our ground-based search for LRDs at $2\lesssim z\lesssim4$, combining ultra-deep Hyper Suprime-Cam photometry and various (near-)infrared surveys within a total area of $\sim3.1\,\mathrm{deg^{2}}$. We find that for LRDs with $M_{5500}<-22.5$, their number density declines from $\sim10^{-4.5}\,\mathrm{cMpc^{-3}}$ at $z>4$ to $\sim10^{-5.3}\,\mathrm{cMpc^{-3}}$ at $2.7<z<3.7$ and $\sim10^{-5.7}\,\mathrm{cMpc^{-3}}$ at $1.7<z<2.7$. We also present the Magellan/FIRE spectrum of our first followed-up candidate, DEEP23-z2LRD1 at $z_\mathrm{spec}=2.26$, as a proof of concept for our sample selection. Similar to high-redshift LRDs, the spectrum of DEEP23-z2LRD1 exhibits broad H$\alpha$ emission with $\mathrm{FWHM}\approx2400\,\mathrm{km\,s^{-1}}$ and with nearly symmetric narrow H$\alpha$ absorption. Additionally, DEEP23-z2LRD1 has extremely narrow [OIII] lines with $\mathrm{FWHM}\approx140\,\mathrm{km\,s^{-1}}$, suggesting the presence of an accreting black hole in a low-mass host galaxy. Limited by the angular resolution of ground-based surveys, we emphasize that spectroscopic follow-ups are required to characterize the contamination fraction of this sample and pin down LRD number density at $z<4$.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.08067-b31b1b.svg)](https://arxiv.org/abs/2504.08067) | **Disk-jet coupling across the spectral transition in supermassive black holes**  |
|| J.-L. Kang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-04-14*|
|*Comments*| *13 pages, 7 figures, submitted to MNRAS. Comments are very welcome!*|
|**Abstract**|            Accretion flows in both stellar and supermassive black holes show a distinct spectral transition. This is seen directly in binaries and changing look AGN, and also in a recent sample of eROSITA X-ray selected, unobscured AGN where the stacked spectral energy distributions (SEDs) for a single black hole mass bin (log $M/M_{\odot} =8-8.5$) clearly show the UV bright disk appearing as the luminosity increases. In binaries, this transition is associated with a change in radio jet, from coupling to the X-ray hot flow with $L_R \propto L_X^{0.7}$ (Fundamental Plane relation), to collapsing when the X-ray hot flow collapses into a disc. We explore the radio behaviour across the transition in our AGN sample by stacking VLASS images. We significantly detect weak radio emission even after subtracting the contribution from star formation in the host galaxy. The residual radio emission remains relatively constant across the transition, despite the mean mass accretion rate changing by a factor 6 and UV flux changing by a factor 100. However, the X-rays change by only a factor 2, giving a constant radio to X-ray flux ratio as predicted by the 'fundamental plane'. We show that this is consistent with these AGN having the same compact radio jet coupling to the X-ray hot flow (not the disc) as in the binaries. The most significant difference is the persistence of the coronal X-rays across the spectral transition in AGN, whereas in binaries the coronal X-rays can be very weak in the disc dominated state.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.08387-b31b1b.svg)](https://arxiv.org/abs/2504.08387) | **Beetroots: spatially-regularized Bayesian inference of physical parameter maps -- Application to Orion**  |
|| P. Palud, et al. -- incl., <mark>A. Hughes</mark> |
|*Appeared on*| *2025-04-14*|
|*Comments*| **|
|**Abstract**|            The current generation of millimeter receivers is able to produce cubes of 800 000 pixels by 200 000 frequency channels to cover several square degrees over the 3 mm atmospheric window. Estimating the physical conditions of the interstellar medium (ISM) with an astrophysical model on such datasets is challenging. Common approaches tend to converge to local minima and typically poorly reconstruct regions with low signal-to-noise ratio (S/N). This instrumental revolution thus calls for new scalable data analysis techniques. We present Beetroots, a Python software that performs Bayesian reconstruction of maps of physical conditions from observation maps and an astrophysical model. It relies on an accurate statistical model, exploits spatial regularization to guide estimations, and uses state-of-the-art algorithms. It also assesses the ability of the astrophysical model to explain the observations, providing feedback to improve ISM models. We demonstrate the power of Beetroots with the Meudon PDR code on synthetic data, and then apply it to estimate physical condition maps in the full Orion molecular cloud 1 (OMC-1) star forming region based on Herschel molecular line emission maps. The application to the synthetic case shows that Beetroots can currently analyse maps with up to ten thousand pixels, addressing large variations of S/N, escaping from local minima, and providing consistent uncertainty quantifications. On a laptop, the inference runtime ranges from a few minutes for 100-pixel maps to 28 hours for 8100-pixel maps. The results on the OMC-1 maps are consistent with independent estimations from the literature, and improve our understanding of the region. This work paves the way towards systematic and rigorous analyses of observations produced by current and future instruments.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.08436-b31b1b.svg)](https://arxiv.org/abs/2504.08436) | **Bifurcated Evolutionary Pathways in Multi-planet Systems Driven by Misaligned Protoplanetary Disks**  |
|| T. Fu, <mark>Y. Wang</mark>, W. Hu |
|*Appeared on*| *2025-04-14*|
|*Comments*| *16 pages, 11 figures, submitted to The Astrophysical Journal Letters (ApJL)*|
|**Abstract**|            Stellar obliquities, or spin-orbit angles, prevalent in exoplanet systems, can impose important constraints on their formation and evolution histories. Recent studies suggest that primordial misalignments between protoplanetary disks and stellar spin axes may significantly contribute to these obliquities, as those frequently observed in systems hosting hot Jupiters. In this study, we demonstrate that misaligned protoplanetary disks combined with stellar oblateness drive complex dynamical evolution in planetary systems during their disk dispersal stages. Specifically, we identify bifurcated evolutionary pathways in multi-planet systems: systems with low star-disk misalignment angles ($\psi_{\star0}$) undergo smooth, adiabatic evolution, producing nearly coplanar, low-obliquity configurations; in contrast, systems with high misalignment angles typically experience an abrupt, non-adiabatic transition, leading to large-amplitude libration of mutual planetary inclinations and then triggering chaotic eccentricity excitation. This libration and eccentricity excitation process can propagate inward-outward in compact multi-planet systems, forming an excitation chain that can destabilize the entire system. The non-adiabatic transition arises from the dynamical bifurcation-induced effect, which occurs during disk dissipation when $\psi_{\star0}\gtrsim44.6^\circ$ (for one-planet systems). Our framework predicts that surviving typical compact multi-planet systems originating from misaligned disks evolve toward coplanar, low-obliquity configurations, consistent with observations of Kepler multi-planet systems. These results advance our understanding of planetary dynamics in misaligned disks and their evolutionary outcomes.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2504.08029/./DQTau_lc_acc.png', 'tmp_2504.08029/./MIRI_lines_HIseries.png', 'tmp_2504.08029/./DQTau_relation_alt.png']
copying  tmp_2504.08029/./DQTau_lc_acc.png to _build/html/
copying  tmp_2504.08029/./MIRI_lines_HIseries.png to _build/html/
copying  tmp_2504.08029/./DQTau_relation_alt.png to _build/html/
exported in  _build/html/2504.08029.md
    + _build/html/tmp_2504.08029/./DQTau_lc_acc.png
    + _build/html/tmp_2504.08029/./MIRI_lines_HIseries.png
    + _build/html/tmp_2504.08029/./DQTau_relation_alt.png
found figures ['tmp_2504.08034/./hist_m_l_paper_test4.png', 'tmp_2504.08034/./m_l_vs_lum_paper_test_new5.png', 'tmp_2504.08034/./paper_fig_3.jpg']
copying  tmp_2504.08034/./hist_m_l_paper_test4.png to _build/html/
copying  tmp_2504.08034/./m_l_vs_lum_paper_test_new5.png to _build/html/
copying  tmp_2504.08034/./paper_fig_3.jpg to _build/html/
exported in  _build/html/2504.08034.md
    + _build/html/tmp_2504.08034/./hist_m_l_paper_test4.png
    + _build/html/t

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\url}[1]{\href{#1}{#1}}$
$\newcommand{\dodoi}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{\doeprint}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{\doarXiv}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{\teff}{\ensuremath{T_{\text{eff}}}}$
$\newcommand{\logg}{log(g)}$
$\newcommand{\vmac}{v_{\rm mac}}$
$\newcommand{\hal}{H\alpha}$
$\newcommand{\fbol}{F_{\mathrm{bol}}}$
$\newcommand{\rsun}{R_\odot}$
$\newcommand{\rstar}{R_\star}$
$\newcommand{\mstar}{M_\star}$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\lsun}{L_\odot}$
$\newcommand{\rhosun}{\rho_\odot}$
$\newcommand{\degs}{^\circ}$
$\newcommand{\funit}{erg s^{-1} cm^{-2} Å^{-1}}$
$\newcommand{\vsini}{vsini}$
$\newcommand{\htwo}{H_2}$
$\newcommand{\lam}{\lambda}$
$\newcommand{\mdot}{\dot{M}}$
$\newcommand{\hei}{He {\scshape i}}$
$\newcommand{\hi}{H {\scshape i}}$
$\newcommand{\oi}{[O {\scshape i}]}$
$\newcommand{\caii}{Ca {\scshape ii}}$
$\newcommand{\lii}{Li {\scshape i}}$
$\newcommand{\nai}{Na {\scshape i}}$
$\newcommand{\ari}{Ar {\scshape i}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\gaia}{{\it Gaia}}$
$\newcommand{\hst}{{\it HST}}$
$\newcommand{\kepler}{{\it Kepler}}$
$\newcommand{\ktwo}{{\it K}2}$
$\newcommand{\tess}{{\it TESS}}$
$\newcommand{\Tess}{{\it TESS}}$
$\newcommand{\spitzer}{{\it Spitzer}}$
$\newcommand{\Spitzer}{{\it Spitzer}}$
$\newcommand{\jwst}{{\it JWST}}$
$\newcommand$
$\newcommand{\fspot}{f_{\rm s}}$
$\newcommand{\fspotecl}{f_{\rm s, ecl}}$
$\newcommand{\co}{^{12}{\mathrm{CO}}  J=2-1}$
$\newcommand{\lacc}{L_{\rm acc}}$
$\newcommand{\up}{{\it u}^\prime}$
$\newcommand{\rp}{{\it r}^\prime}$
$\newcommand{\AB}[1]{\textcolor{orange}{\textbf{(AB:} #1)}}$
$\newcommand{\CFM}[1]{\textcolor{purple}{\textbf{(CFM:} #1)}}$
$\newcommand{\BT}[1]{\textcolor{red}{\textbf{(BT:} #1)}}$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# Coordinated Space and Ground-Based Monitoring of Accretion Bursts in a Protoplanetary Disk: \ Establishing Mid-Infrared Hydrogen Lines as Accretion Diagnostics for $\jwst$--MIRI

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.08029-b31b1b.svg)](https://arxiv.org/abs/2504.08029)<mark>Appeared on: 2025-04-14</mark> -  _Accepted to ApJ. 16 pages, 7 Figures, 3 Tables, 2 Appendices_

</div>
<div id="authors">

B. M. Tofflemire, et al. -- incl., <mark>M. Benisty</mark>

</div>
<div id="abstract">

**Abstract:** In this paper, we establish and calibrate mid-infrared hydrogen recombination lines observed with $\jwst$ as accretion tracers for pre-main-sequence stars that accrete from circumstellar disks. This work is part of a coordinated, multi-observatory effort that monitored the well-known binary system DQ Tau over three orbital periods, capturing its periodic accretion bursts.In this first paper, we present 9 epochs of MIRI-MRS spectra with near-simultaneous LCO photometry and VLT--X-Shooter spectroscopy. This program caught exceptional accretion variability, spanning almost two orders of magnitude between the peak of the first periastron accretion burst and the following quiescent phases. The MIRI spectra show $\hi$ line luminosities that vary in step with the accretion-luminosity time series measured with LCO and X-Shooter. The tight correlation with accretion and the large line widths, which MIRI resolves for the first time, support an accretion-flow origin for mid-infrared $\hi$ transitions. Combining these three exceptional datasets, we derive accurate relations between mid-infrared line and accretion luminosities for three $\hi$ transitions (10-7, 7-6, 8-7), and improve upon a previous relation based on Spitzer spectra. These new relations equip the community with a direct measurement of the accretion luminosity from MIRI-MRS spectra. A MIRI-derived accretion luminosity is fundamental for time-domain chemistry studies, as well as for studies of accretion in embedded/distant sources that are currently inaccessible in the optical. With these new relations, we provide accretion luminosities for an archival sample of 38 MRS spectra of protoplanetary disks published to date.

</div>

<div id="div_fig1">

<img src="tmp_2504.08029/./DQTau_lc_acc.png" alt="Fig4" width="100%"/>

**Figure 4. -** DQ Tau accretion variability. ** Top:** The accretion luminosity (left axis) and mass accretion rate (right axis) from X-Shooter spectra and LCO $\up$-band photometry. Time is presented in barycentric Julian days on the bottom axis and the binary orbital cycle on the top axis. Vertical dotted lines mark binary periastron passages. Blue vertical lines mark the times of MIRI-MRS observations with an associated epoch number. The dashed horizontal line represents the accretion luminosity measurement floor for the $\up$ photometry (see Section \ref{lco_lacc}; Figure \ref{fig: lacc_relation}). The gray vertical region surrounding MIRI Epoch 3 highlights a reddening event. The black point is an estimate of the accretion luminosity during this event (see Section \ref{reddening}).
** Bottom:** Variability in the $\hi$(10-7) emission line. Continuum subtracted MIRI-MRS spectra centered on the line are presented for each observation, with the epoch number listed on the top right corner of each panel. The $\hi$(10-7) temporal behavior tightly tracks the densely sampled accretion luminosity measurements. (*fig: lightcurve_epochs*)

</div>
<div id="div_fig2">

<img src="tmp_2504.08029/./MIRI_lines_HIseries.png" alt="Fig5" width="100%"/>

**Figure 5. -** Gallery of some strong hydrogen recombination lines observed in MIRI spectra. Two DQ Tau epochs are shown in each panel to illustrate a high accretion phase (September 11th, in solid black) and the lowest accretion phase in this program (October 7th, dashed black). A hot water model (850 K) is shown in blue for reference. The (10-7) and (8-7) lines are selected from those identified in \cite{banzatti24} as free of contamination from molecular emission. Two examples of significant contamination from water are shown for reference: the (7-6) line \citep[used in][]{rigliaco2015} and the (6-5) line, the strongest $\hi$ line observable with MIRI-MRS, but contaminated with ro-vibrational water emission. The vertical gray dashed lines indicate the ranges where the line flux is measured, in the case of the (7-6) line to minimize contamination from the nearby (11-8) line.  (*fig: HI_series*)

</div>
<div id="div_fig3">

<img src="tmp_2504.08029/./DQTau_relation_alt.png" alt="Fig6" width="100%"/>

**Figure 6. -** Correlations between $\hi$ line luminosities and $\lacc$ as measured in this monitoring program. The $\hi$(7-6) luminosity has been corrected for water contamination as explained in Section \ref{acc_relations}. Large colored data points are from the DQ Tau epochs in this work. Smaller green circles show JDISCS and MINDS published spectra with non-contemporaneous literature $\lacc$ measurements (see text). The larger scatter is likely due to non-synchronous measurements and varied $\lacc$ measurement methodologies. For non detections, the 2-$\sigma$ upper limit is shown with an arrow. The black dashed lines show the median correlations as measured for DQ Tau, while the orange line in the middle panel shows the $\hi$(7-6) relation previously measured by \cite{rigliaco2015}. The offset with the new relation found from MIRI is due to the limited correction from contamination that was possible with IRS (see Section \ref{acc_relations}). (*fig: HI_Lacc_correlations*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.08029"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\tbd}[1]{{\color{red} #1}}$</div>



<div id="title">

# X-ray Constraints on Wandering Black Holes in Stripped Galaxy Nuclei in the Halo of NGC 5128

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.08034-b31b1b.svg)](https://arxiv.org/abs/2504.08034)<mark>Appeared on: 2025-04-14</mark> -  _13 pages, 3 figures, 2 tables. Accepted for publication in ApJ_

</div>
<div id="authors">

S. L. Feyan, et al. -- incl., <mark>A. Dumont</mark>

</div>
<div id="abstract">

**Abstract:** A subset of galaxies have dense nuclei, and when these galaxies are accreted and tidally stripped, the nuclei can masquerade as globular clusters in the halos of large galaxies. If these nuclei contain massive central black holes, some may accrete gas and become observable as active galactic nuclei. Previous studies have found that candidate stripped nuclei rarely host luminous X-ray sources, but these studies were typically restricted to both the most massive candidate nuclei and the most luminous X-ray sources. Here we use new and archival Chandra and XMM-Newton data to search for X-ray emission in a near-complete sample of massive globular clusters and candidate stripped nuclei in the nearest accessible elliptical galaxy, NGC 5128. This sample has the unique advantage that the candidate stripped nuclei are identified dynamically via elevated mass-to-light ratios. Our central result is that 5/22 ( $23^{+11}_{-6}$ \% ) of the candidate stripped nuclei have X-ray sources down to a typical limit of $L_X \sim 5 \times 10^{36}$ erg s $^{-1}$ , a fraction lower than or comparable to that among massive clusters with normal mass-to-light ratios (16/41; $39^{+8}_{-7}$ \% ). Hence we confirm and extend the result that nearly all X-ray sources in stripped nuclei are likely to be X-ray binaries rather than active galactic nuclei. If the candidate stripped nuclei have black holes of typical masses $\sim 2 \times 10^{5} M_{\odot}$ needed to explain their elevated mass-to-light ratios, then they have typical Eddington ratios of $\lesssim 2 \times 10^{-6}$ . This suggests that it will be challenging to conduct an accretion census of wandering black holes around even nearby galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2504.08034/./hist_m_l_paper_test4.png" alt="Fig1" width="100%"/>

**Figure 1. -** Histogram of $M/L_V$ for all 63 objects (dashed line) and X-ray detections (shaded). The bimodal distribution of $M/L_V$ is visible in both samples. (*fig:ml_hist*)

</div>
<div id="div_fig2">

<img src="tmp_2504.08034/./m_l_vs_lum_paper_test_new5.png" alt="Fig2" width="100%"/>

**Figure 2. -** 1--10 keV X-ray luminosity vs. $M/L_V$ for all sources, with detections shown as filled circled and non-detections (upper limits) shown as unfilled downward triangles. The similarity of the X-ray properties of the low and high-$M/L_V$ objects is evident. There are three sources with $M/L_V$ much larger than the other clusters, which have especially strong dynamical cases for being stripped nuclei. (*fig:ml_plot*)

</div>
<div id="div_fig3">

<img src="tmp_2504.08034/./paper_fig_3.jpg" alt="Fig3" width="100%"/>

**Figure 3. -** The 63 clusters in our sample plotted on a Digitized Sky Survey image of Cen A, with north up and east to the left. While many clusters are projected within the central 10s of kpc of the galaxy, some are in the distant outer halo, with the southernmost object at 116 kpc projected. The 30 kpc scale bar corresponds to $0.45^{\circ}$. (*fig:spat*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.08034"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\TODO}[1]{\textcolor{magenta}{\textsc{todo:} \textit{#1}}}$
$\newcommand{\TODOall}[1]{\textcolor{red}{\textsc{Task/Question to all:} \textit{#1}}}$
$\newcommand{\prelim}[1]{\textcolor{magenta}{\textit{#1}}}$
$\newcommand{\CRX}{CRX}$
$\newcommand{\dLW}{dLW}$
$\newcommand{\CONTRAST}{CCF-contrast}$
$\newcommand{\FWHM}{CCF-FWHM}$
$\newcommand{\BISECTOR}{CCF-BIS}$
$\newcommand{\CaHtwo}{CaH~2}$
$\newcommand{\CaHthree}{CaH~3}$
$\newcommand{\CaIRT}{Ca {\sc ii}~IRT}$
$\newcommand{\CaIRTa}{Ca~IRT_\text{a}}$
$\newcommand{\CaIRTb}{Ca~IRT_\text{b}}$
$\newcommand{\CaIRTc}{Ca~IRT_\text{c}}$
$\newcommand{\Fe}{Fe~\lambda\SI{8689}{\angstrom}}$
$\newcommand{\Halpha}{H\alpha}$
$\newcommand{\He}{He~{\sc i} \lambda\SI{10830}{\angstrom}}$
$\newcommand{\HeDthree}{He {\sc i}~D_3}$
$\newcommand{\NaDdoublet}{Na {\sc i} doublet}$
$\newcommand{\NaDone}{Na~D_1}$
$\newcommand{\NaDtwo}{Na~D_2}$
$\newcommand{\Pabeta}{Pa\beta}$
$\newcommand{\TiOsevenzero}{TiO~\lambda\SI{7048}{\angstrom}}$
$\newcommand{\TiOeightfour}{TiO~\lambda\SI{8428}{\angstrom}}$
$\newcommand{\TiOeighteight}{TiO~\lambda\SI{8858}{\angstrom}}$
$\newcommand{\VOsevenfour}{VO~\lambda\SI{7434}{\angstrom}}$
$\newcommand{\VOsevenine}{VO~\lambda\SI{7940}{\angstrom}}$
$\newcommand{\WFB}{FeH WFB}$
$\newcommand{\autoref}$
$\newcommand{\inst}[1]{\unskip^{\instrefs{#1}}}$
$\newcommand{\equationautorefname}{Eq.}$
$\newcommand{\figureautorefname}{Fig.}$
$\newcommand{\sectionautorefname}{Sect.}$
$\newcommand{\subsectionautorefname}{Sect.}$
$\newcommand{\subsubsectionautorefname}{Sect.}$
$\newcommand{\scsep}{\newcommand\scsep{,}}$
$\newcommand\scsep{,}$</div>



<div id="title">

# The CARMENES search for exoplanets around M dwarfs: Cluster analysis of signals from spectral activity indicators to search for shared periods

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.08363-b31b1b.svg)](https://arxiv.org/abs/2504.08363)<mark>Appeared on: 2025-04-14</mark> -  _33 pages (16 pages main text), 14 figures, accepted for publication in Astronomy & Astrophysics_

</div>
<div id="authors">

J. Kemmer, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** A multitude of spectral activity indicators are  routinely computed nowadays from the spectra generated as part of planet-hunting radial velocity surveys. Searching for shared periods among them can help to robustly identify astrophysical quantities of interest, such as the stellar rotation period. However, this identification can be complicated due to the fact that many different peaks occurring in the periodograms. This is especially true in the presence of aliasing and spurious signals caused by environmental influences affecting the instrument. Our goal is to test a clustering algorithm to find signals with the same periodicity, (i.e. with the stellar rotation period) in the periodograms of a large number of activity indicators. On this basis, we have looked to evaluate the correlations between activity indicators and fundamental stellar parameters. We used generalised Lomb-Scargle periodograms to find periodic signals in 24 activity indicators, spanning the VIS and NIR channels of the CARMENES spectrograph.Common periods were subsequently determined by a machine learning algorithm for density-based spatial clustering of applications with noise ( ${\tt DBSCAN}$ ). The clustering analysis of the signals apparent in the spectral activity indicators is a powerful tool for the detection of stellar rotation periods.It is straightforward to implement and can be easily automated, so that large data sets can be analysed. For a sample of 136 stars, we were able to recover the stellar rotation period in a total of 59 cases, including 3 with a previously unknown rotation period. In addition, we analysed spurious signals frequently occurring at the period of one year and its integer fractions,  concluding that they are likely aliases of one underlying signal. Furthermore, we reproduced the results of several previous studies on the relationships between activity indicators and the stellar characteristics.

</div>

<div id="div_fig1">

<img src="tmp_2504.08363/./figures/srp/activity_indicators_clusters_J02530+168_naive.png" alt="Fig10.1" width="33%"/><img src="tmp_2504.08363/./figures/srp/activity_indicators_clusters_J02530+168_prewhitened.png" alt="Fig10.2" width="33%"/><img src="tmp_2504.08363/./figures/srp/activity_indicators_clusters_J02530+168_prewhitened_clean.png" alt="Fig10.3" width="33%"/>

**Figure 10. -** Results of the clustering algorithm for Teegarden's Star. _Top_: Using the naive approach. _Middle_: Using the pre-whitening approach. _Bottom_: Using pre-whitening and excluding periods close to the harmonics of one year. The coloured filled symbols show the individual data points of the detected clusters, with the error bars corresponding to the peak width in the GLS, and the open symbols yield the mean values of the clusters. Grey dots denote data points that were identified as noise by the clustering algorithm.
  The colour mapping changes between the plots to emphasise that although there is some overlap, the clusters (and thus technically also the mean periods) that appear  change between the different methods. Clusters that are consistent between all plots within the GLS resolution are always denoted by the same non-circular markers. The FAP of \SI{1}{\percent} is marked by the grey horizontal dotted line and the grey vertical dashed lines show different periods of interest. The baseline indicates the timespan of the observations. (*fig:clustering_teegarden*)

</div>
<div id="div_fig2">

<img src="tmp_2504.08363/./figures/srp/srp_hist_matches_rotation_period_spt.png" alt="Fig12" width="100%"/>

**Figure 12. -** Histogram showing how often  individual indicators were part of  clusters that matched  stellar rotation periods from the literature (either directly or in a harmonic or alias relation). The different colours show the total counts in each bar, differentiated by stellar spectral type. We distinguish between inactive and active (pEW($\Halpha$) $<\SI{-0.5}{\angstrom}$) stars by using solid and hatched bars respectively. (*fig:period_matches_per_indicator*)

</div>
<div id="div_fig3">

<img src="tmp_2504.08363/./figures/srp/srp_recovery_rate_spt.png" alt="Fig11.1" width="50%"/><img src="tmp_2504.08363/./figures/srp/srp_no_cluster_spt.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** Recovery rate and fraction of non-detections of stellar rotation period as a function of stellar spectral type. _Left_: Fraction of stars for which the literature periods were recovered (either directly or in a harmonic or alias relation). _Right_: Fraction of stars that show no significant clusters (including stars without a literature rotation period). In the plots we distinguish between inactive and active (pEW($\Halpha$) $<\SI{-0.5}{\angstrom}$) stars by using solid and hatched bars, respectively. In the left panel, the number on top of each bar denotes the number of stars with a known rotation period for each category. In the right panel, it yields the overall number of stars in the selected sample.
    For better readability, we chose normal black font for the inactive sample and italic grey font for the active sample. (*fig:comparison_spt*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.08363"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

438  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
